In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("pdf_rag")

LangSmith 추적을 시작합니다.
[프로젝트명]
pdf_rag


## RAG 기본 파이프라인

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings

In [12]:
# 1 단계: 문서 로드(Load Document)
loader = PyMuPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

# 2 단계: 문서 분할(Split Document)
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

split_documents = text_spliter.split_documents(docs)



# 3 단계: 문서 임베딩(Embed Document) 생성
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 4 단계: DB 생성(Create DB)및 저장
# pc 공간에 저장 가능
vectorstore = FAISS.from_documents(documents = split_documents, embedding=hf)

/Users/insu/Library/Caches/pypoetry/virtualenvs/langchain-rag-UhYHSyIM-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
docs[1].page_content

'2023년 12월호\nⅠ. 인공지능 산업 동향 브리프\n 1. 정책/법제 \n   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1\n   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2\n   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3\n   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4\n   ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5\n   ▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··················································· 6\n \n 2. 기업/산업 \n   ▹ 미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성································ 7\n   ▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8\n   ▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9\n   ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10\n   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 

In [14]:
docs[1].metadata

{'source': '../data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': '../data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 1,
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13462',
 'producer': 'Hancom PDF 1.3.0.542',
 'creationDate': "D:20231208132838+09'00'",
 'modDate': "D:20231208132838+09'00'",
 'trapped': ''}

In [15]:
# 5 단계: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성.
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x325e8dfd0>)

In [16]:
retriever.invoke("삼성전자가 개발한 생성형 AI 이름은 무엇인가요?")

[Document(page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,', metadata={'source': '../data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': '../data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12, 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.13462', 'producer': 'Hancom PDF 1.3.0.542', 'creationDate': "D:20231208132838+09'00'", 'modDate': "D:20231208132838+09'00'", 'trapped': ''}),
 Document(page_content='▹ EU AI 법 3자 협상, 기반모델 규제 관련 견해차로 난항··········

In [18]:
# 6단계: 프롬프트 생성(Create Prompt)
prompt = PromptTemplate.from_template(
    """ You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer ther question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [19]:
# 7단계: 언어모델(LLM) 생성(Create LLM)
llm = ChatOllama(
    model = 'llama3',
    temperature= 0
)

# 8단계 체인 생성(Create Chain)
chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [20]:
question = "삼성에서 새로 개발한 생성형 AI의 이름은 무엇인가요?"
answer = chain.invoke("삼성에서 새로 개발한 생성형 AI의 이름은 무엇인가요?")
print(answer)

삼성 가우스(Samsung Gauss)입니다.
